# Components, mixtures and phase equilibrium model basics

In this notebook basic creation of pure components, fluid mixtures and the available models in Phasepy are exemplified.

Phasepy unit for temperature is [K], unit of pressure is [bar] and unit of molar volume is [cm^3/mol].

- [Single components](#pure-fluids)
- [Mixtures](#fluid-mixtures)
- [Models](#models)
    - [Discontinuous Models](#gamma-phi)
        - [NRTL](#nrtl)
        - [Wilson](#wilson)
        - [Redlich-Kister Expansion](#redlich-kister)
        - [UNIQUAC](#uniquac)
        - [Modified UNIFAC (Dortmund)](#unifac)
    - [Cubic Equations of State](#cubic-eos)
        - [QMR mixing rule](#qmr-mixingrule)
        - [MHV mixing rule](#mhv-mixingrule)
        - [MHV1 mixing rule](#mhv1-mixingrule)
        - [Wong-Sandler mixing rule](#ws-mixingrule)

In [1]:
import numpy as np
from phasepy import mixture, component, virialgamma, preos

<a id='pure-fluids'></a>

## Single components

Physical properties are specified when a component is created. Typical properties include critical properties (``Tc``, ``Pc``, ``Zc``, ``Vc``), acentric factor (``w``), Antoine Coefficients (``Ant``), group specification for Modified UNIFAC group contribution (``GC``) method and molecular volume and surface (``ri`` and ``qi``) used in UNIQUAC.

The following Antoine's equation is implemented in the `component` class.

$$ \ln (P /bar) = A - \frac{B}{T/K + C}$$ 

Where $P$ is the saturation pressure and $T$ is the absolute temperature. The parameters $A$, $B$ and $C$ are obtained as follows: `A, B, C = component.Ant`.

The liquid volume of the fluids can be estimated using Rackett's equation:

$$ v = v_c Z_c^{(1 - T_r)^{2/7}} $$ 

Where, $v$ is the liquid volume, $v_c$ is the critical volume, $Z_c$ is the critical fluid compressibility and $T_r$ is the reduced temperature. You need to provide the critical parameter (`Tc`, `Vc` and`Zc`) for this method to work.

In [2]:
water = component(name='water', Tc=647.13, Pc=220.55, Zc=0.229, Vc=55.948, w=0.344861,
                  Ant=[11.64785144, 3797.41566067, -46.77830444],
                  GC={'H2O':1}, ri=0.92, qi=1.4)

ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    Ant=[11.61809279, 3423.0259436, -56.48094263],
                    GC={'CH3':1, 'CH2':1, 'OH(P)':1}, ri=2.1055, qi=1.972)

print('Water saturation pressure at 373 K:', water.psat(T=373.0), 'bar')
print('Water liquid molar volume at 310 K:', water.vlrackett(T=310.0), 'cm3/mol')
print('Ethanol saturation pressure at 373 K:', ethanol.psat(T=373.0), 'bar')
print('Ethanol liquid molar volume at 310 K:', ethanol.vlrackett(T=310.0), 'cm3/mol')

Water saturation pressure at 373 K: 1.0072796747419537 bar
Water liquid molar volume at 310 K: 16.46025809309672 cm3/mol
Ethanol saturation pressure at 373 K: 2.233335305328437 bar
Ethanol liquid molar volume at 310 K: 56.32856995891473 cm3/mol


In [3]:
# Example 1. Peng Robinson EOS for single component (without volume translation)
eos = preos(ethanol)
print('Saturation pressure, liquid and vapor molar volumes:', eos.psat(T=350.0))
print('Liquid density:', eos.density(T=350.0, P=1.0, state='L'))
print('Vapor density:', eos.density(T=350.0, P=1.0, state='V'))

Saturation pressure, liquid and vapor molar volumes: (array([0.98800647]), array([66.75754804]), array([28799.31921623]))
Liquid density: 0.01497960198094922
Vapor density: 3.515440899573752e-05


In [4]:
# Example 2. Peng Robinson EOS for single component (with volume translation, parameter c)
ethanol = component(name='ethanol', Tc=514.0, Pc=61.37, Zc=0.241, Vc=168.0, w=0.643558,
                    c=5.35490936, Ant=[11.61809279, 3423.0259436, -56.48094263],
                    GC={'CH3':1, 'CH2':1, 'OH(P)':1}, ri=2.1055, qi=1.972)
eos = preos(ethanol, volume_translation=True)
print('Saturation pressure, liquid and vapor molar volumes:', eos.psat(T=350.0))
print('Liquid density:', eos.density(T=350.0, P=1.0, state='L'))
print('Vapor density:', eos.density(T=350.0, P=1.0, state='V'))

Saturation pressure, liquid and vapor molar volumes: (array([0.98800647]), array([61.40263868]), array([28793.96430687]))
Liquid density: 0.01628597159790686
Vapor density: 3.5161028012629526e-05


<a id='fluid-mixtures'></a>

## Mixtures

Mixtures are created from components with the ``mixture`` class function or by adding components (`+`).

The mixture function allows setting interaction parameters, such for NRTL model (``mixture.NRTL``), Wilson model (``mixture.wilson``), Redlich-Kister polynomial (``mixture.rk``), UNIQUAC model (``mixture.uniquac``) and UNIFAC model (``mixture.unifac``).

In [5]:
# creating a mixture with the mixture class function
mix = mixture(ethanol, water)
# creating a mixture by adding two components
mix = ethanol + water

print('Component names:', mix.names)
print('Number of components:', mix.nc)
print('Component saturation pressures:', mix.psat(T=373.0))
print('Component liquid molar volumes:', mix.vlrackett(T=310.0))

Component names: ['ethanol', 'water']
Number of components: 2
Component saturation pressures: [2.23333531 1.00727967]
Component liquid molar volumes: [56.32856996 16.46025809]


Phasepy can handle multicomponent mixtures. You can add a component to an existing mixture using the `add_component` method or by adding it to the mixture (`+`). This is shown below.

In [28]:
mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 Ant=[9.16238246, 2541.97883529, -50.40534341],
                 GC={'CH3':3, 'CH3O':1, 'C':1})

# adding a component using the add_component method
mix = mixture(ethanol, water)
mix.add_component(mtbe)

# creating the ternary mixture by adding the pure components
mix = ethanol + water + mtbe

# adding a component to an existing mixture
mix = mixture(ethanol, water)
mix += mtbe

<a id='models'></a>

## Phase equilibrium models

Phase equilibrium models are created from a mixture and model specifications. Model-specific parameters must be provided to the mixture.

Phasepy includes two modeling approaches. In the discontinuous modeling approach, where the vapor can be modeled from a virial expansion and the liquid is modeled using an activity coefficient model. In the continuous modeling approach, both vapor al liquid phases are modeled with the same equation of state.


<a id='gamma-phi'></a>
## Discontinous modeling:

Available models through the `virialgamma` function.

 - Vapor phase: `Abbott`, `Tsonopoulos` or `ideal_gas`.
 - Liquid phase: `nrtl`, `wilson`, `rk`, `uniquac` or `unifac`.




<a id='nrtl'></a>
### NRTL activity coefficient model

The NRTL model computes the Gibbs's excess energy as follows:

$$ g^e = \sum_{i=1}^c  x_i \frac{\sum_{j=1}^c \tau_{ji}G_{ji}x_j}{\sum_{l=1}^c G_{li}x_l}$$

Where, 
$$ \tau_{ji} = \frac{A_{ji}}{T}, \quad G_{ji} = \exp{(-\alpha_{ji}\tau_{ji})}, \quad 
\alpha_{ji} = \alpha_{ij} $$

Here $A_{ij}$ and $\alpha_{ij}$ are the interaction energy and aleatory factor between the component $i$ and $j$

In Phasepy, the NRTL's interaction energies are computed as: $A_{ij} = A_{ij, 0} + A_{ij, 1} T$.

You can supply the NRTL parameter using the `mix.NRTL` method.

In [7]:
# Example 3. Initialization of NRTL activity coefficient model for liquid, and Abbott virial model for vapor
mix = ethanol + water

alpha = np.array([[0.0, 0.5597628],
                  [0.5597628, 0.0]])
Aij0 = np.array([[0.0, -57.6880881],
              [668.682368, 0.0]])
Aij1 = np.array([[0.0, 0.46909821],
               [-0.37982045, 0.0]])
mix.NRTL(alpha, Aij0, Aij1)
eos = virialgamma(mix, virialmodel='Abbott', actmodel='nrtl')

<a id='wilson'></a>
### Wilson activity coefficient model


In the Wilson's model the Gibbs's excess energy is computed as follows:

$$ g^e = - \sum_i x_i \ln\left[ \sum_j  x_j \Lambda_{ij}\right]$$

Where,

$$ \Lambda_{ij} = \frac{v_j}{v_i} \exp{\frac{-A_{ij}}{T}}$$

Here, $v_i$ is volume of the component $i$ and $A_{ij}$ is the interaction energy between component $i$ and $j$. The energies can be fitted to experimental data. Phasepy uses Rackett Equation to compute volumes in Wilson's model, you need to provide critical properties of the fluid for this method to work (`Tc`, `Pc`, `Zc`, `Vc`, `w`).

You can supply the Wilson energies using the ``mix.wilson`` method.

In [8]:
# Example 5. Initialization of Wilson activity coefficient model for liquid, and Abbott virial model for vapor
mix = ethanol + water

Aij = np.array([[0., 163.97284376],
                [497.26813303, 0.]])
mix.wilson(Aij)
eos = virialgamma(mix, virialmodel='Abbott', actmodel='wilson')

<a id='redlich-kister'></a>
### Redlich Kister Expansion activity coefficient model

he Redlich-Kister expansion model the Gibbs's excess energy as a polynomial:

$$ g^e_{ij} = x_ix_j \sum_{k=0}^m C_k (x_i - x_j)^k $$

In Phasepy polynomials coefficients are computed as:  $C_k = C_0 + C_1/T$

You can supply the Redlich-Kister polynomial coefficients using the ``mix.rk`` method.

In [9]:
# Example 6. Inititalization of Redlich Kister expansion for liquid, Tsonopoulos virial model for vapor
mix = ethanol + water

C0 = np.array([1.20945699, -0.62209997, 3.18919339])
C1 = np.array([-13.271128, 101.837857, -1100.29221])
mix.rk(C0, C1)
eos = virialgamma(mix, virialmodel='Tsonopoulos', actmodel='rk')

<a id='uniquac'></a>
### UNIQUAC activity coefficient model

In the UNIQUAC model, the activity coefficients are computed as the sum of a combinatory and a residual term.

$$\ln \gamma_i = \underbrace{ \ln \frac{\Phi_i}{x_i} + 1 \frac{\Phi_i}{x_i} - \frac{10}{2} q_i \left[ \ln \frac{\Phi_i}{\vartheta_i} + 1 - \frac{\Phi_i}{\vartheta_i} \right] }_{\ln \gamma_i^{comb}} + \underbrace{q_i \left[1 - \ln S_i - \sum_i \frac{\tau_{ij}\vartheta_j}{S_j} \right]}_{\ln \gamma_i^{res}} $$

Where,

$$ \Phi_i = \frac{r_i x_i}{\sum_j r_j x_j} \quad \vartheta_i = \frac{q_i x_i}{\sum_j q_j x_j} \quad \tau_{ij} = \exp \left[\frac{A_{ij}}{T}\right] \quad S_i = \sum_j \vartheta_j \tau_{ji} $$

The UNIQUAC activiy coefficient model requires the molecular volume ($r_i$) and surface ($q_i$) as the interaction energy ($A_{ij}$).  In Phasepy the interaction energy is computed as follows: $A_{ij} = A_{ij, 0} + A_{ij, 1} T$.

**note1:** you need to provide the molecular surface and volume (``ri`` and ``qi``) to the components for this method to work.

**note2:** you can supply the UNIQUAC energies using the ``mix.uniquac`` method.


In [10]:
# Example 6. Inititalization of UNIQUAC for liquid, ideal gas law for vapor
mix = ethanol + water

# interactions energies in K
A12, A21 = -7.33540823, 151.83100234 
Aij0 = np.array([[0., A12], [A21, 0.]])
mix.uniquac(Aij0)
eos = virialgamma(mix, virialmodel='ideal_gas', actmodel='uniquac')

<a id='unifac'></a>
### Modified-Dortmund UNIFAC activity coefficient model

In the Dortmund-UNIFAC group contribution activity model, the activity coefficients are obtained from a combinatorial and a residual contribution.

$$\ln \gamma_i = \ln \gamma_i^{comb} + \ln \gamma_i^{res} $$


Phasepy includes the public parameters published [here](http://www.ddbst.com/PublishedParametersUNIFACDO.html).

You need to provide the group contribution information through the `GC` parameter when creating a component for this method to work.

Once your mixture has been set up, you can read the database using the `mix.unifac` method.

In [11]:
# Example 7. Inititalization of Dortmund Modified UNIFAC for liquid, ideal gas law for vapor
mix = ethanol + water

mix.unifac()
eos = virialgamma(mix, virialmodel='ideal_gas', actmodel='unifac')

<a id='cubic-eos'></a>
### Cubics Equations of State

Phasepy includes the following cubic equations of state: `vdweos`, `rkeos`, `rkseos`, `preos`, `prsveos`. When working with mixtures you can use the classical quadratic mixing rule or choose from one of the implemented advanced mixing rules (MHV, MHV-1, or WS).

<a id='qmr-mixingrule'></a>
#### Quadratic mixing rule (QMR)

In quadratic mixing rule (`qmr`) the cohesive parameter and covolume is computed as follows:

$$ b_{mix} = \sum_{i} x_i b_i \qquad a_{mix} = \sum_i \sum_j x_i x_j \sqrt{a_i a_j} (1-k_{ij}) $$

You can supply the ``Kij`` correction matrix using the ``mix.kij_cubic`` method.

In [12]:
# Example 8. Initialization of Peng Robinson EOS for mixture with Quadratic mixing rule.
mix = ethanol + water

Kij = np.array([[0, -0.11], [-0.11, 0]])
mix.kij_cubic(Kij)
eos = preos(mix, mixrule='qmr')

<a id='mhv-mixingrule'></a>

#### Modified Huron-Vidal  mixing rule (MHV)

The Modified Huron-Vidal mixing rule matches the Gibbs excess energy from an activity coefficient model and the obtained from the cubic EoS at the low pressure limit ($P \rightarrow 0$). With this mixing rule the covolume is obtained as $ b_{mix} = \sum_{i} x_i b_i$. While the cohesive parameter is obtained from the solution to the following non-linear equation:

$$ g^{E, model} + \sum_i x_i \ln \left[\frac{b_{mix}}{b_i} \right] = q(\alpha) - \sum_i x_i q(\alpha_i)$$

Where, $\alpha = a/(b RT)$ and the $q(\alpha)$ function is computed as follows:

$$ q(\alpha) = -1 - \ln(u_0 - 1) - \frac{\alpha}{c_1-c_2} \ln \left[\frac{u_0 + c_1}{u_0 + c_2} \right] $$

Where $u_0 = V/b$ is computed from the volume solution at zero pressure from the cubic equation of state.

**note1:** This set of equations has a solution ($\alpha$) only at low pressures, you should not use this mixing rule at pressures higher than ~ 5 bar.

**note2:** You can couple this mixing rule with any of the implemented activity coefficient models. You need to provide the model's parameter to the `mixture` object.

In [13]:
# Example 10. Initialization of Peng Robinson EOS for mixture. Mixing rule is Modified Huron Vidal with NRTL.
mix = ethanol + water

alpha = np.array([[0.0, 0.5597628],
                  [0.5597628, 0.0]])
Aij0 = np.array([[0.0, -57.6880881],
              [668.682368, 0.0]])
Aij1 = np.array([[0.0, 0.46909821],
               [-0.37982045, 0.0]])
mix.NRTL(alpha, Aij0, Aij1)
eos = preos(mix, mixrule='mhv_nrtl')

In [14]:
# Example 11. Initialization of Peng Robinson EOS for mixture. Mixing rule is Modified Huron Vidal with Wilson.
mix = ethanol + water

# interaction energies in #K
Aij = np.array([[0., 163.97284376],
                [497.26813303, 0.]])
mix.wilson(Aij)
eos = preos(mix, mixrule='mhv_wilson')

In [15]:
# Example 12. Initialization of Peng Robinson EOS for mixture. Mixing rule is Modified Huron Vidal with Redlich Kister.
mix = ethanol + water

C0 = np.array([1.20945699, -0.62209997, 3.18919339])
C1 = np.array([-13.271128, 101.837857, -1100.29221])
mix.rk(C0, C1)
eos = preos(mix, mixrule='mhv_rk')

In [16]:
# Example 13. Initialization of Peng Robinson EOS for mixture. Mixing rule is Modified Huron Vidal with UNIQUAC.
mix = ethanol + water
# note: you need to provide the molecular surface and volume (ri and qi) to the components for this method to work.

# interactions energies in K
A12, A21 = -7.33540823, 151.83100234 
Aij0 = np.array([[0., A12], [A21, 0.]])
mix.uniquac(Aij0)
eos = preos(mix, mixrule='mhv_uniquac')

In [17]:
# Example 14. Initialization of Peng Robinson EOS for mixture. Mixing rule is Modified Huron Vidal with Modified UNIFAC.
mix = ethanol + water

mix.unifac()
eos = preos(mix, mixrule='mhv_unifac')

<a id='mhv1-mixingrule'></a>


#### Modified Huron-Vidal-1 mixing rule (MHV-1)

The original MHV mixing rule can be used only at low pressure. Michelsen proposed a linearization of the $q(\alpha)$ function.

$$ q(\alpha) \approx q_0 + q_1 \alpha \rightarrow  \alpha = \frac{1}{q_1} \left[ g^{E, model} + \sum_i x_i \ln \frac{b_{mix}}{b_i} \right] + \sum_i x_i \alpha_i $$

Where the $q_1$ parameter has been fitted to experimental data for PR, PRSK, vdW EoS. This mixing rule is known as MHV-1.

**note:** You can couple this mixing rule with any of the implemented activity coefficient models. You need to provide the model's parameter to the `mixture` object.

In [18]:
# Example 15. Initialization of Peng Robinson EOS for mixture. Mixing rule is MHV1 with NRTL.
mix = ethanol + water

alpha = np.array([[0.0, 0.5597628],
                  [0.5597628, 0.0]])
Aij0 = np.array([[0.0, -57.6880881],
              [668.682368, 0.0]])
Aij1 = np.array([[0.0, 0.46909821],
               [-0.37982045, 0.0]])
mix.NRTL(alpha, Aij0, Aij1)
eos = preos(mix, mixrule='mhv1_nrtl')

In [19]:
# Example 16. Initialization of Peng Robinson EOS for mixture. Mixing rule is MHV1 with Wilson.
mix = ethanol + water

Aij = np.array([[0., 163.97284376],
                [497.26813303, 0.]])
mix.wilson(Aij)
eos = preos(mix, mixrule='mhv1_wilson')

In [20]:
# Example 17. Initialization of Peng Robinson EOS for mixture. Mixing rule is MHV1 with Redlich Kister.
mix = ethanol + water

C0 = np.array([1.20945699, -0.62209997, 3.18919339])
C1 = np.array([-13.271128, 101.837857, -1100.29221])
mix.rk(C0, C1)
eos = preos(mix, mixrule='mhv1_rk')

In [21]:
# Example 18. Initialization of Peng Robinson EOS for mixture. Mixing rule is MHV1 with UNIQUAC.
mix = ethanol + water
# note: you need to provide the molecular surface and volume (ri and qi) to the components for this method to work.

# interactions energies in K
A12, A21 = -7.33540823, 151.83100234 
Aij0 = np.array([[0., A12], [A21, 0.]])
mix.uniquac(Aij0)
eos = preos(mix, mixrule='mhv1_uniquac')

In [22]:
# Example 19. Initialization of Peng Robinson EOS for mixture. Mixing rule is MHV1 with Modified UNIFAC.

mix = ethanol + water
mix.unifac()

eos = preos(mix, mixrule='mhv1_unifac')

<a id='ws-mixingrule'></a>

#### Wong-Sandler mixing rule (WS)

This advanced mixing rule uses the infinity pressure reference and considers a theoretically correct calculation of the second-virial:

$$ B = \sum_i \sum_j x_i x_j \left[ b - \frac{a}{RT} \right]_{ij} = b_{mix} - \frac{a_{mix}}{RT} $$

In this mixing rule, the covolume is computed as follows:

$$ b_{mix}= \frac{\sum_i \sum_j x_i x_j \left[ b - \frac{a}{RT} \right]_{ij}}{1 - \frac{g^{E, model}}{C^*} - \sum_i x_i \frac{a_i}{b_i RT}} $$

This allows an adjustable parameter ``kij`` for each pair. 

$$ \left[ b - \frac{a}{RT} \right]_{ij} = \frac{1}{2} \left[ \left( b_i - \frac{a_i}{RT} \right) + \left( b_j - \frac{a_j}{RT} \right) \right] \left(1 -k_{ij} \right)$$

Finally, the cohesive parameter is computed as follows:

$$ a_{mix} = b_{mix} RT \left[ \frac{g^{E, model}}{C^*} + \sum_i x_i \frac{a_i}{b_i RT} \right]$$

**note1:** you can supply the ``Kij`` correction matrix using the ``mix.kij_ws`` method.


**note2:** You can couple this mixing rule with any of the implemented activity coefficient models. You need to provide the model's parameter to the `mixture` object.

In [23]:
# Example 20. Initialization of Peng Robinson EOS for mixture. Mixing rule is WS with NRTL.
mix = ethanol + water

alpha = np.array([[0.0, 0.5597628],
                  [0.5597628, 0.0]])
Aij0 = np.array([[0.0, -57.6880881],
              [668.682368, 0.0]])
Aij1 = np.array([[0.0, 0.46909821],
               [-0.37982045, 0.0]])
mix.NRTL(alpha, Aij0, Aij1)
eos = preos(mix, mixrule='ws_nrtl')

In [24]:
# Example 21. Initialization of Peng Robinson EOS for mixture. Mixing rule is WS with Wilson.
mix = ethanol + water


Aij = np.array([[0., 163.97284376],
                [497.26813303, 0.]])
mix.wilson(Aij)
eos = preos(mix, mixrule='ws_wilson')

In [25]:
# Example 22. Initialization of Peng Robinson EOS for mixture. Mixing rule is WS with Redlich Kister.
mix = ethanol + water

C0 = np.array([1.20945699, -0.62209997, 3.18919339])
C1 = np.array([-13.271128, 101.837857, -1100.29221])
mix.rk(C0, C1)
eos = preos(mix, mixrule='ws_rk')

In [26]:
# Example 23. Initialization of Peng Robinson EOS for mixture. Mixing rule is WS with UNIQUAC.
mix = ethanol + water
# note: you need to provide the molecular surface and volume (ri and qi) to the components for this method to work.

# interactions energies in K
A12, A21 = -7.33540823, 151.83100234 
Aij0 = np.array([[0., A12], [A21, 0.]])
mix.uniquac(Aij0)
eos = preos(mix, mixrule='ws_uniquac')

In [27]:
# Example 24. Initialization of Peng Robinson EOS for mixture. Mixing rule is WS with Modified UNIFAC.
mix = ethanol + water

mix.unifac()

kijws = 0.226140922855718
Kij_ws = np.array([[0, kijws], [kijws, 0.]])

mix.kij_ws(Kij_ws)

eos = preos(mix, mixrule='ws_unifac')

For further information please also check [official documentation](https://phasepy.readthedocs.io/), or just try:

```function?```